In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=a09acba9ab2b4a5a78fd2f601e3905069d8e266b4c20447b7d20335e6af82838
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 4.9MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.33 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_7_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_7_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0,10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1029264, Tue Apr 14 16:38:11 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.015256718407938e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.970920226387075e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21538.52897804] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21838.77355529] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.879414725292672e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   29    551 8.345838609456261e+03 1.1e+00 2.04e+00  2e+00  2e+00 0:03.4
   64   1216 5.576863293721577e+03 1.2e+00 1.89e+00  2e+00  2e+00 0:07.5
  100   1900 -3.694984691876089e+02 1.2e+00 1.42e+00  1e+00  1e+00 0:11.7
  151   2869 -9.335003566547875e+03 1.2e+00 9.06e-01  9e-01  9e-01 0:17.7
  200   3800 -1.653452689524007e+04 1.2e+00 5.46e-01  5e-01  6e-01 0:23.5
  270   5130 -2.035077695948301e+04 1.2e+00 2.57e-01  3e-01  3e-01 0:31.6
  300   5700 -2.097390684908741e+04 1.2e+00 1.87e-01  2e-01  2e-01 0:35.1
  386   7334 -2.158468353495883e+04 1.2e+00 7.65e-02  8e-02  8e-02 0:45.1
  400   7600 -2.161465168041770e+04 1.2e+00 6.69e-02  7e-02  7e-02 0:46.7
  500   9500 -2.169857101755673e+04 1.2e+00 2.25e-02  2e-02  2e-02 0:58.2
  600  11400 -2.170611041299991e+04 1.2e+00 6.89e-03  7e-03  7e-03 1:09.9
  700  13300 -2.170686742554995e+04 1.3e+00 2.41e-03  2e-03  2e-03 1:21.4
  800  15200 -2.170695675177687e+04 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [23153.52062679] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23153.520627, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23153.520627, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Tue Apr 14 16:59:37 2020)
final/bestever f-value = 2.315352e+04 2.315352e+04
incumbent solution: [ 2.01783739 -1.77093941  2.22812409 -2.10096131  4.01576908  0.05934078
 -4.45534991  2.41953322 ...]
std deviations: [2.43675894 2.43664489 2.43694779 2.43666523 2.43692792 2.43714027
 2.43639023 2.43714082 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1034023, Tue Apr 14 16:59:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.044805383473824e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 2.012549104114094e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.3


...

    2     38 2.350164401827256e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.3
    3     57 2.231855772751222e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   26    494 7.912397824684596e+03 1.1e+00 2.13e+00  2e+00  2e+00 0:03.4
   57   1083 6.268622784008410e+03 1.1e+00 2.00e+00  2e+00  2e+00 0:07.5
   95   1805 2.716257567448247e+03 1.2e+00 1.62e+00  2e+00  2e+00 0:12.5
  100   1900 4.182839498629146e+02 1.2e+00 1.56e+00  2e+00  2e+00 0:13.1
  154   2926 -9.756415844053499e+03 1.2e+00 9.19e-01  9e-01  9e-01 0:20.1
  200   3800 -1.582740958626443e+04 1.2e+00 5.66e-01  6e-01  6e-01 0:26.1
  270   5130 -1.997583804793670e+04 1.2e+00 2.67e-01  3e-01  3e-01 0:35.1
  300   5700 -2.089838871190376e+04 1.2e+00 2.01e-01  2e-01  2e-01 0:39.3
  385   7315 -2.158398173850982e+04 1.2e+00 8.07e-02  8e-02  8e-02 0:50.4
  400   7600 -2.161050436456970e+04 1.2e+00 6.75e-02  7e-02  7e-02 0:52.4
  500   9500 -2.169583796638623e+04 1.2e+00 2.30e-02  2e-02  2e-02 1:05.5
  600  11400 -2.170566979782588e+04 1.3e+00 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [23153.52062679] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23153.520627, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23153.520627, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Wed Apr 15 00:04:45 2020)
final/bestever f-value = 2.315352e+04 2.315352e+04
incumbent solution: [-0.70078968  1.34519421  4.91246024 -0.73510379 -2.50325149  4.94664789
 -1.7909843  -2.2878028  ...]
std deviations: [2.43678793 2.4371851  2.43685353 2.43684427 2.43683702 2.4369287
 2.43697935 2.43699637 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=975148, Wed Apr 15 00:04:45 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.592044946483038e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 2.359200866935579e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [30627.7773194] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [25046.77662908] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.985700654429451e+04 1.0e+00 2.35e+00  2e+00  2e+00 0:00.4
   28    532 6.820088873290624e+02 1.1e+00 2.05e+00  2e+00  2e+00 0:03.4
   61   1159 -5.758157737962159e+03 1.1e+00 1.83e+00  2e+00  2e+00 0:07.5
  100   1900 -1.011789732001509e+04 1.2e+00 1.42e+00  1e+00  1e+00 0:12.2
  150   2850 -1.648727353021255e+04 1.2e+00 1.09e+00  1e+00  1e+00 0:18.3
  200   3800 -2.189247015834500e+04 1.3e+00 7.78e-01  8e-01  8e-01 0:24.4
  264   5016 -2.652321810864192e+04 1.3e+00 4.86e-01  5e-01  5e-01 0:32.5
  300   5700 -2.920358657056996e+04 1.3e+00 3.63e-01  4e-01  4e-01 0:36.9
  381   7239 -3.156981452047055e+04 1.3e+00 2.00e-01  2e-01  2e-01 0:47.0
  400   7600 -3.172439283591250e+04 1.3e+00 1.82e-01  2e-01  2e-01 0:49.3
  498   9462 -3.272413604630335e+04 1.3e+00 8.92e-02  9e-02  9e-02 1:01.4
  500   9500 -3.274817410212884e+04 1.3e+00 8.89e-02  9e-02  9e-02 1:01.6
  600  11400 -3.308185644316100e+04 1.4e+00 6.75e-02  7e-02  7e-02 1:13.9
  700  13300 -3.325891168665257e+04 1.6e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F7_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F7(X_Values.iloc[i,:200])
    SVM_Fun [i] = F7(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F7(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([4151.53796328]), count=array([10])) 0.0
SVM
ModeResult(mode=array([17890.7127378]), count=array([1])) 2804.4665232318325
ELN
ModeResult(mode=array([25035.23851038]), count=array([2])) 1076.8436791187194
